# Introducción

En un mercado de telecomunicaciones altamente competitivo, la retención de clientes y la optimización de sus planes de suscripción son elementos clave para garantizar la sostenibilidad y el crecimiento de las empresas. Megaline, una compañía líder en servicios móviles, enfrenta el desafío de modernizar a sus clientes que todavía utilizan planes heredados, menos eficientes y alineados con sus objetivos comerciales actuales. Para abordar este problema, Megaline busca implementar un modelo basado en datos que analice el comportamiento de los clientes y recomiende, con precisión, uno de los nuevos planes disponibles: Smart o Ultra.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier

In [3]:
df=pd.read_csv('/datasets/users_behavior.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


In [4]:
print(df.head())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


El conjunto de datos cuenta con 3214 entradas y 5 columnas: calls, minutes, messages, mb_used, y is_ultra. Todas las columnas tienen 3214 valores no nulos, lo que significa que no hay valores faltantes en los datos. Las primeras cuatro columnas (calls, minutes, messages, y mb_used) son de tipo float64, mientras que la columna is_ultra, que parece ser la variable objetivo (0 o 1), es de tipo int64.

# Características (x) - Variable Objetivo (y) 

In [19]:
x = df.drop(columns=['is_ultra'])
y = df['is_ultra']

# Datos fuente en un conjunto de entrenamiento, validación y de prueba.

In [20]:
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.4, random_state=42)

x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

#  Calidad de diferentes modelos

In [21]:
param_tree = {
    'max_depth':[3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
} 

param_forest = {
    'n_estimators':[50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2']
}

param_logistic = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver':['liblinear', 'lbfgs'],
    'max_iter': [500, 1000]
}

grid_tree = GridSearchCV(DecisionTreeClassifier(random_state = 42), param_tree, cv=5)
grid_forest = GridSearchCV(RandomForestClassifier(random_state=42), param_forest, cv=5)
grid_logistic = GridSearchCV(LogisticRegression(random_state=42), param_logistic, cv=5)

grid_tree.fit(x_train, y_train)
grid_forest.fit(x_train, y_train)
grid_logistic.fit(x_train, y_train)

valid_acc_tree = accuracy_score(y_valid, grid_tree.best_estimator_.predict(x_valid))
valid_acc_forest = accuracy_score(y_valid, grid_forest.best_estimator_.predict(x_valid))
valid_acc_logistic = accuracy_score(y_valid, grid_logistic.best_estimator_.predict(x_valid))

print(f'Árbol de Decisión - Mejor exactitud: {valid_acc_tree:.4f}')
print(f'Bosque Aleatorio - Mejor exactitud: {valid_acc_forest:.4f}')
print(f'Regresión Logística - Mejor exactitud:{valid_acc_logistic:.4f}')

Árbol de Decisión - Mejor exactitud: 0.7916
Bosque Aleatorio - Mejor exactitud: 0.8025
Regresión Logística - Mejor exactitud:0.7185


Los resultados del estudio muestran que, entre los tres modelos probados (Árbol de Decisión, Bosque Aleatorio y Regresión Logística), el Bosque Aleatorio ofrece la mejor exactitud con un valor de 0.8025. Esto significa que es el modelo más eficaz para predecir correctamente el plan adecuado para los clientes, superando el umbral de exactitud de 0.75 solicitado.

Árbol de Decisión: Alcanzó una exactitud de 0.7916, lo cual es competitivo pero ligeramente inferior al Bosque Aleatorio.
Bosque Aleatorio: Obtuvo la mayor exactitud con 0.8025, mostrando que agregar múltiples árboles mejora las predicciones y reduce el sobreajuste.
Regresión Logística: Alcanzó una exactitud de 0.7185, la más baja de los tres modelos, lo cual sugiere que este enfoque es menos eficaz en este contexto.

# Calidad del modelo usando el conjunto de prueba

In [22]:
test_acc_forest = accuracy_score(y_test, grid_forest.best_estimator_.predict(x_test))

print(f'Bosque Aleatorio - Exactitud en el conjunto de prueba: {test_acc_forest:.2f}')

Bosque Aleatorio - Exactitud en el conjunto de prueba: 0.81


Bosque Aleatorio fue el mejor modelo con una exactitud de 0.8025 en la validación y 0.81 en el conjunto de prueba. Este modelo demuestra que es capaz de generalizar bien sobre datos que no ha visto antes, superando el umbral de exactitud de 0.75.

#  Prueba de cordura al modelo

Modelo trivial (predicción de la clase mayoritaria)

In [23]:
dummy_majority = DummyClassifier(strategy='most_frequent')
dummy_majority.fit(x_train, y_train)

dummy_majority_acc = accuracy_score(y_test, dummy_majority.predict(x_test))
print(f'Modelo trivial (clase más frecuente) - Exactitud: {dummy_majority_acc:.2f}')

Modelo trivial (clase más frecuente) - Exactitud: 0.70


Modelo aleatorio

In [24]:
dummy_random = DummyClassifier(strategy='uniform')
dummy_random.fit(x_train, y_train)

dummy_random_acc = accuracy_score(y_test,dummy_random.predict(x_test))
print(f'Modelo Aleatorio - Exactitud: {dummy_random_acc:.2f}')

Modelo Aleatorio - Exactitud: 0.51


La prueba de cordura revela que el modelo de Bosque Aleatorio está funcionando de manera efectiva, superando tanto al modelo aleatorio como al trivial. Esto sugiere que el modelo ha aprendido patrones significativos a partir de los datos y es capaz de hacer predicciones mucho más precisas que los enfoques triviales.

# Conclusión

En este proyecto, se desarrolló un modelo de clasificación para la compañía Megaline con el fin de recomendar uno de sus planes nuevos: Smart o Ultra. Se probaron varios algoritmos, como Árbol de Decisión, Bosque Aleatorio y Regresión Logística, con el objetivo de alcanzar una exactitud mínima de 0.75. Tras segmentar los datos y ajustar los hiperparámetros, el Bosque Aleatorio resultó ser el modelo más eficaz, alcanzando una exactitud de 0.81 en el conjunto de prueba, superando las expectativas iniciales.

Además, se realizaron pruebas de cordura utilizando un modelo trivial (predicción de la clase mayoritaria) y un modelo aleatorio, demostrando que el Bosque Aleatorio supera significativamente estos enfoques simples. Este resultado muestra que el modelo es capaz de identificar patrones reales en los datos de comportamiento de los usuarios, proporcionando una herramienta confiable para que Megaline optimice la recomendación de sus planes de suscripción, lo que puede traducirse en una mejor retención de clientes y una mayor satisfacción del servicio.